# Developing RASPA agent

In [20]:
from latex_parsing import *

def parse_tex(filename):
    with open(filename) as f:
            latex_text = f.read()
    return construct_tree(split_latex_sections(latex_text, depth=0))

In [21]:
def parse_node(node):
    return (node.title, node.content)

## Knowledge

In [22]:
a='''
TODO:
- Latex parsing
- Annotation
    - Delete irrelevant parts
    - Annotate tool capabilities
'''

### RASPA theory

In [23]:
a='''
1.1 Design Philosophy
1.2. Units and conventions

2. Format of the Input Files
    2.1. Introduction
    2.2. Simulation input
    (2.3. Force field) -> automatically with trappe tool
    (2.4. Molecules) -> automatically with trappe tool
    (2.5. Framework) -> coremof or other tool (for zeolites, ... TODO)
'''

In [24]:
intro = "raw_knowledge/introduction.tex"
intro_parsed = parse_tex(intro)

philosophy = intro_parsed[0]
philosophy = parse_node(philosophy)

units = intro_parsed[1].children[0]
units = parse_node(units)

In [25]:
input_files = "raw_knowledge/input_files.tex"
input_files_parsed = parse_tex(input_files)

introduction = input_files_parsed[0]
sim_input = input_files_parsed[1]
# ff = input_files_parsed[2]
# molecules = input_files_parsed[3]
# framework = input_files_parsed[4]

In [26]:
# TODO

In [27]:
def filter():
    # remove irrelevant parts
    pass

def annotate():
    # highlight tool capabilities
    pass

### RASPA input/output examples - annotated

In [28]:
a='''
4.2 Basic examples
partially 
    4.3 Non-basic examples
    4.4 Advanced examples
    4.5 Auxiliary examples
partially 4.6 
    Number of cycles and run-times
'''

In [29]:
examples = "raw_knowledge/examples.tex"
examples_parsed = parse_tex(examples)

general = examples_parsed[0].content[:223]
files = examples_parsed[0].children

sim = parse_node(files[0])
ff = parse_node(files[1])
mol = parse_node(files[2])
structure = parse_node(files[3])

## Questions / Tasks

### General questions

In [30]:
q1 = "How to setup a RASPA simulation and how do you use your tools for it?"
q2 = "How to find the critical temperature of a molecule?"
q3 = "What types of simulations can you run and which types of properties can you calculate?"
q4 = "How do you calculate the gas storage potential for a MOF?"
q5 = ""
q6 = ""
q7 = ""
q8 = ""

### Simulation tasks

In [31]:
s1 = "Provide simulation files for running an energy minimization for CO2 in NU-1000"
s2 = "Find me a structure that has Fm-3m topology and then run methane adsorption in it"
s3 = "please tell me the performance of reinsertion and partial reinsertion moves for benzene adsorption in MFI type zeolites"
s4 = "Run a tertiary mixture adsorption of hexane and its isomers in MFI zeolite at 298K and pressures 0 - 100 Pa"
s5 = "Create simulation files for finding a minimum energy location of methane in MFI zeolite?"
s6 = "Please provide simulation files for running a adsorption simulation of methane in IFMOF-1"
s7 = "Provide simulation input files to find the Henry law coefficient of methane in NU-1000"
s8 = "Provide Input files for running GEMC for methane in NU-1000"
s9 = "simulate dynamic behaviour of hexane in MFI zeolite"
s10 = "Run simulation to find the critical temperature of heptane"

### Output analysis tasks

In [32]:
o1 = "What is the Henry coefficient?"
o2 = "Compare the simulations for heptane and pentane. Which one has a higher ...?"
o3 = ""
o4 = ""
o5 = ""
o6 = ""
o7 = ""
o8 = ""
o9 = ""
o10 = ""

### Tutorial Tasks

In [33]:
tutorial = "raw_knowledge/tutorial.tex"
tutorial_parsed = parse_tex(tutorial)

In [34]:
t1 = parse_node(tutorial_parsed[0])
t2 = parse_node(tutorial_parsed[1])
t3 = parse_node(tutorial_parsed[2])

In [35]:
# TODO: extract "exercises"

## Teaching Workflow

In [3]:
a='''Try RASPAagent.run() vs explicit MemoryAgent.learn() -> maybe RaspaAgent.learn() ?

for knowledge:
    while response needs no clarification:
        response = agent.run()
        answer = "human response"
        response = agent.run(answer)
'''


In [1]:
from student.agent.memory import Memory
# from agent_input_file import init_input_file_memory
# examples_memory = init_input_file_memory()
examples_memory = Memory()
examples_memory.load("input_file_memory.json")
examples_memory.recall(["Monte Carlo"])

{'fe02d599': '\n        <memory id="fe02d599">\n            <stimuli>Monte Carlo, Monte Carlo of methane in a box</stimuli>\n            <content># Monte Carlo of methane in a box\nSimulationType                MonteCarlo\nNumberOfCycles                10000\nNumberOfInitializationCycles  5000\nPrintEvery                    1000\n\nForcefield                    ExampleMoleculeForceField\n\n\nBox 0\nBoxLengths 30 30 30\nExternalTemperature 300.0\nMovies yes\nWriteMoviesEvery 100\n\nComponent 0 MoleculeName             methane\n            MoleculeDefinition       ExampleDefinitions\n            TranslationProbability   1.0\n            CreateNumberOfMolecules  100</content>\n        </memory>\n        ',
 '97486aae': '\n        <memory id="97486aae">\n            <stimuli>Monte Carlo of a binary mixture in a box, Monte Carlo</stimuli>\n            <content># Monte Carlo of a binary mixture in a box\nSimulationType                MonteCarlo\nNumberOfCycles                10000\nNumberOfI